In [1]:
import pymssql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from pandas import DataFrame, to_datetime
from datetime import date, timedelta
from prophet import Prophet
import datetime

def DB_connection(tank_seq):
    conn = pymssql.connect(server='sql16ssd-014.localnet.kr', user='i2on11_admin', password='root0826', database='i2on11_admin')
    cursor = conn.cursor()

    conn_sql = 'SELECT signal_time, tank_remain_volume FROM gas_tank_volume_history where tank_seq = ' + tank_seq + 'ORDER BY signal_time DESC'

    # 쿼리 실행
    cursor.execute(conn_sql)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 연결 닫기
    conn.close()
    import datetime
    days_ago = datetime.datetime.now().date() - datetime.timedelta(days=30)
    days_ago_datetime = datetime.datetime.combine(days_ago, datetime.datetime.min.time())

    d_h = pd.DataFrame(rows, columns=['ds', 'y'])
    d_h['ds'] = pd.to_datetime(d_h['ds'])
    d_h.set_index('ds', inplace=True)
    d_h.sort_index(ascending=True, inplace=True) # O(n) 100
    d_h = d_h[d_h['y'] > 5] # n개의 탐색시간을 가지고 n 번 행위를 하는 O(n) 100
    d_h = d_h[d_h.index >= days_ago_datetime] # O(n) 100

    diff_d_h = d_h.copy() # O(n)
    diff_d_h['y'] = diff_d_h['y'].diff()
    diff_d_h = diff_d_h.dropna()

    try:
        recent_datetime = diff_d_h[diff_d_h['y'] > 15].index.max()
        fd_h = d_h[d_h.index > recent_datetime]
#         if len(fd_h)<3 and len(d_h)!=0:
#             try:
#                 second_largest_datetime = diff_d_h[diff_d_h['y'] > 15].index.nlargest(2).iloc[-1]
#                 d_h = d_h[d_h.index > second_largest_datetime ]
#             except:
#                 d_h = fd_h
#         else :
#             d_h = fd_h
    except:
        pass

    d_h.reset_index(inplace=True)
    diff_d_h.reset_index(inplace=True)
    
    return d_h, diff_d_h

In [2]:
def prophet_prediction(d_h, diff_d_h):
    # Prophet 객체 생성
    model = Prophet()

    # 데이터로 모델 훈련
    model.fit(d_h)

    # 향후 15일 동안의 예측
    future = model.make_future_dataframe(periods=15)
    forecast = model.predict(future)
    forecast['ds'] = pd.to_datetime(forecast['ds'])
    dh_index_max = d_h['ds'].max()
    latest_forecast = forecast[forecast['ds'] >= pd.to_datetime(dh_index_max)]
    
    prediction = dict(zip(latest_forecast['ds'], latest_forecast['yhat']))
    return prediction


In [3]:
def prediction_by_tank_seq(tank_seq):
    d_h, diff_d_h = DB_connection(tank_seq)
    
    pred_h = prophet_prediction(d_h, diff_d_h)
    return pred_h

#prediction_by_tank_seq('327')


In [ ]:
from flask import request, render_template, Markup
from pandarallel import pandarallel
import pandas as pd
import time
from flask import Flask
from multiprocessing.dummy import Pool as ThreadPool
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

pandarallel.initialize()
app = Flask(__name__)
executor = concurrent.futures.ThreadPoolExecutor()

@app.route('/test')
def test_parallel_route():
    data = request.form
    exception = []
    start_time = time.time()

    tank_list = ['327', '338', '759', '347', '323', '437', '358', '396', '547', '503', '353', '512', '370', '327', '371']
    runningvalue = Markup('<div> tank_list_length : ' + str(len(tank_list)) + '</div>')
    
    def run_prophet(tank_seq):
        return prediction_by_tank_seq(tank_seq)
    
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(run_prophet, tank_list))
    
    for tank_seq, result in zip(tank_list, results):
        try:
            runningvalue += Markup('<div> tank_seq : ' + str(tank_seq))
            for key, value in result.items():
                runningvalue += Markup('<div>' + str(key) + ', ' + str(value) + '</div>')
            runningvalue += Markup('</div>')
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port="5000")

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.21:5000/ (Press CTRL+C to quit)
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] start processing
14:13:23 - cmdstanpy - INFO - Chain [1] done processing
14:13:24 - cmdstanpy - INFO - Chain [1] done processin

14:16:49 - cmdstanpy - INFO - Chain [1] done processing
14:16:49 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:16:50] "GET /test HTTP/1.1" 200 -
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] start processing
14:16:50 - cmdstanpy - INFO - Chain [1] done processing
14:16:50 - cmdstanpy - INFO

14:17:09 - cmdstanpy - INFO - Chain [1] done processing
14:17:09 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:17:09] "GET /test HTTP/1.1" 200 -
14:17:17 - cmdstanpy - INFO - Chain [1] start processing
14:17:17 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INFO - Chain [1] start processing
14:17:18 - cmdstanpy - INF

14:17:25 - cmdstanpy - INFO - Chain [1] done processing
14:17:25 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:17:25] "GET /test HTTP/1.1" 200 -
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] done processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO - Chain [1] start processing
14:17:26 - cmdstanpy - INFO

...

14:18:16 - cmdstanpy - INFO - Chain [1] done processing
14:18:16 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:18:16] "GET /test HTTP/1.1" 200 -
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INFO - Chain [1] start processing
14:18:18 - cmdstanpy - INF

14:18:23 - cmdstanpy - INFO - Chain [1] done processing
14:18:23 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:18:23] "GET /test HTTP/1.1" 200 -
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INFO - Chain [1] start processing
14:18:24 - cmdstanpy - INF

14:18:39 - cmdstanpy - INFO - Chain [1] done processing
14:18:40 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:18:40] "GET /test HTTP/1.1" 200 -
14:18:40 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] done processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO - Chain [1] start processing
14:18:41 - cmdstanpy - INFO

14:19:22 - cmdstanpy - INFO - Chain [1] done processing
14:19:22 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:19:23] "GET /test HTTP/1.1" 200 -
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INFO - Chain [1] start processing
14:19:23 - cmdstanpy - INF

14:19:28 - cmdstanpy - INFO - Chain [1] done processing
14:19:28 - cmdstanpy - INFO - Chain [1] done processing
127.0.0.1 - - [13/Jun/2023 14:19:29] "GET /test HTTP/1.1" 200 -
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INF